In [ ]:
# analysis.ipynb
# Preliminary basic data analysis

# Immuno-Oncology Clinical Trials Analysis from https://clinicaltrials.gov/
# Immuno-Oncology Clinical Trials Analysis from https://eudract.ema.europa.eu/

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import process

In [ ]:
# Data from ClinicalTrials - 05/07/2025 (dd/mm/yyyy)
df1 = pd.read_csv("ctg-studies.csv")  
print(df1.head())

In [ ]:
# Data from EudraCT - 05/07/2025 (dd/mm/yyyy)
df2 = pd.read_csv("ctis-studies.csv") 
print(df2.head())

In [ ]:
# ADDING IDs

df1['id'] = range(1, len(df1) + 1)
df2['id'] = range(len(df1) + 1, len(df1) + len(df2) + 1)


In [ ]:
# LOOKING FOR DUPLICATES

df1['Study Title'] = df1['Study Title'].str.lower()
df2['Title of the trial'] = df2['Title of the trial'].str.lower()
duplicates = []

for i in df2.index:
    trial_title = df2.loc[i, 'Title of the trial']
    trial_id = df2.loc[i, 'id']

    match = process.extractOne(trial_title, df1['Study Title'], score_cutoff=95)

    if match:
        best_title, score, pos = match
        matched_id = df1.loc[pos, 'id']
        matched_title = df1.loc[pos, 'Study Title']
        print(f"\nMATCH FOUND:")
        print(f"  df2 ID {trial_id}  ->  \"{trial_title}\"")
        print(f"  df1 ID {matched_id}  ->  \"{matched_title}\"")
        print(f"  Similarity score: {score}")
        print("---")
        duplicates.append( (trial_id, matched_id) )

# TO BE IMPROVED...

In [ ]:
# MERGING DATA: FINAL DF

# IDs
ids_df2_to_exclude = [ pair[0] for pair in duplicates ]
df2_clean = df2[ ~df2['id'].isin(ids_df2_to_exclude) ]
df = pd.concat([df1[['id']], df2_clean[['id']]],ignore_index=True)

In [ ]:
# ADDING LOCATIONS

df['location'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        loc_string = df1.loc[df1['id'] == trial_id, 'Locations'].values[0]
        
        countries = set()
        if pd.notna(loc_string):
            # separar per |
            sites = loc_string.split("|")
            for site in sites:
                parts = site.strip().split(",")
                if len(parts) > 0:
                    country = parts[-1].strip()
                    countries.add(country)
            # convertir a string
            country_str = ", ".join(sorted(countries)) if countries else None
        else:
            country_str = None
        
        df.loc[idx, 'location'] = country_str
        
    else:
        loc_string = df2_clean.loc[df2_clean['id'] == trial_id, 'Location(s) and recruitment status'].values[0]
        
        if pd.notna(loc_string):
            country = loc_string.strip().split(":")[0].strip()
        else:
            country = None
        
        df.loc[idx, 'location'] = country

In [ ]:
# ADDING MEDICAL CONDITIONS

df['condition'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        cond_string = df1.loc[df1['id'] == trial_id, 'Conditions'].values[0]
        
        if pd.notna(cond_string):
            condition = cond_string.strip()
        else:
            condition = None
            
        df.loc[idx, 'condition'] = condition
        
    else:
        cond_string = df2_clean.loc[df2_clean['id'] == trial_id, 'Medical conditions'].values[0]
        
        if pd.notna(cond_string):
            condition = cond_string.strip()
        else:
            condition = None
            
        df.loc[idx, 'condition'] = condition

In [ ]:
# ADDING INTERVENTIONS

df['intervention'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        intv_string = df1.loc[df1['id'] == trial_id, 'Interventions'].values[0]
        
        if pd.notna(intv_string):
            intervention = intv_string.strip()
        else:
            intervention = None
            
        df.loc[idx, 'intervention'] = intervention
        
    else:
        intv_string = df2_clean.loc[df2_clean['id'] == trial_id, 'Product'].values[0]
        
        if pd.notna(intv_string):
            intervention = intv_string.strip()
        else:
            intervention = None
            
        df.loc[idx, 'intervention'] = intervention

In [ ]:
# ADDING TRIAL PHASES

df['phase'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        phase_string = df1.loc[df1['id'] == trial_id, 'Phases'].values[0]
        
        if pd.notna(phase_string):
            phase_string = phase_string.strip().upper()
            
            if phase_string == "PHASE1":
                phase = 1
            elif phase_string == "PHASE2":
                phase = 2
            elif phase_string == "PHASE3":
                phase = 3
            elif phase_string == "PHASE1|PHASE2":
                phase = 12
            elif phase_string == "PHASE2|PHASE3":
                phase = 23
            else:
                phase = None
                print(f"[ALERTA] Valor desconegut a df1 id {trial_id}: {phase_string}")
        else:
            phase = None
            
        df.loc[idx, 'phase'] = phase
        
    else:
        phase_string = df2_clean.loc[df2_clean['id'] == trial_id, 'Trial phase'].values[0]
        
        if pd.notna(phase_string):
            phase_string = phase_string.strip().lower()
            
            found_phase1 = "phase i" in phase_string
            found_phase2 = "phase ii" in phase_string
            found_phase3 = "phase iii" in phase_string
            
            if found_phase1 and found_phase2:
                phase = 12
            elif found_phase2 and found_phase3:
                phase = 23
            elif found_phase1:
                phase = 1
            elif found_phase2:
                phase = 2
            elif found_phase3:
                phase = 3
            else:
                phase = None
                print(f"[ALERTA] Valor desconegut a df2 id {trial_id}: {phase_string}")
        else:
            phase = None
            
        df.loc[idx, 'phase'] = phase


In [ ]:
# ADDING SPONSORS

# Sponsor classification
df['sponsor_type'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        sponsor_type = df1.loc[df1['id'] == trial_id, 'Funder Type'].values[0]
        df.loc[idx, 'sponsor_type'] = sponsor_type.strip() if pd.notna(sponsor_type) else None
        
    else:
        sponsor_type = df2_clean.loc[df2_clean['id'] == trial_id, 'Sponsor type'].values[0]
        df.loc[idx, 'sponsor_type'] = sponsor_type.strip() if pd.notna(sponsor_type) else None

# Sponsor 
df['sponsor'] = None

ids_df1 = set(df1['id'])

for idx, row in df.iterrows():
    trial_id = row['id']
    
    if trial_id in ids_df1:
        sponsor_values = df1.loc[df1['id'] == trial_id, 'Sponsor'].values
        if len(sponsor_values) > 0 and pd.notna(sponsor_values[0]) and sponsor_values[0].strip() != "":
            sponsor = sponsor_values[0]
        else:
            sponsor = None
    else:
        sponsor_values = df2_clean.loc[df2_clean['id'] == trial_id, 'Sponsor/Co-Sponsors'].values
        if len(sponsor_values) > 0 and pd.notna(sponsor_values[0]) and sponsor_values[0].strip() != "":
            sponsor = sponsor_values[0]
        else:
            sponsor = None
    
    df.loc[idx, 'sponsor'] = sponsor

    # Collaborators

    # PDT


In [ ]:
# SUMMARY OF INTERVENTION FIELD

import re

# DICTIONARIES 
checkpoint_inhibitors = [
    'nivolumab', 'pembrolizumab', 'atezolizumab', 'avelumab', 'durvalumab',
    'cemiplimab', 'tislelizumab', 'camrelizumab', 'toripalimab'
]
ctla4_inhibitors = ['ipilimumab', 'tremelimumab']
parp_inhibitors = ['olaparib', 'niraparib', 'rucaparib', 'talazoparib']
other_targeted = ['lenvatinib', 'sorafenib', 'cabozantinib', 'enzalutamide']
chemo_agents = [
    'carboplatin', 'cisplatin', 'oxaliplatin', 'mitomycin', 'docetaxel',
    'paclitaxel', 'etoposide', 'vincristine', 'cyclophosphamide', 'doxorubicin',
    'gemcitabine', 'irinotecan', 'temozolomide'
]
surgery_terms = ['surgery', 'resection', 'cystectomy', 'prostatectomy', 'nephrectomy', 'tumor removal', 'orchidectomy']
mab_suffix = 'mab'
nib_suffix = 'nib'

manual_map = {
    'xentuzumab': ('antibody', 'mAb'),
    'selpercatinib': ('kinase inhibitor', 'RET inhibitor'),
    'trastuzumab': ('antibody', 'anti-HER2'),
    'bevacizumab': ('antibody', 'anti-VEGF'),
    'avelumab': ('checkpoint inhibitor', 'anti-PD-L1'),
    'lambrolizumab': ('checkpoint inhibitor', 'anti-PD-1'),
    'durvalumab + tremelimumab': ('checkpoint inhibitor', 'combo PD-L1 + CTLA-4'),
    'nivolumab + ipilimumab': ('checkpoint inhibitor', 'combo PD-1 + CTLA-4'),
    'sintilimab': ('checkpoint inhibitor', 'anti-PD-1'),
    'toripalimab': ('checkpoint inhibitor', 'anti-PD-1'),
    'donafenib': ('kinase inhibitor', 'TKI'),
    'camrelizumab': ('checkpoint inhibitor', 'anti-PD-1'),
    'atezolizumab + bevacizumab': ('checkpoint inhibitor', 'combo PD-L1 + anti-VEGF'),
    'pemetrexed': ('chemotherapy', 'antifolate'),
    'fludarabine': ('chemotherapy', 'purine analog'),
    'capecitabine': ('chemotherapy', 'fluoropyrimidine'),
    'opdivo 10': ('checkpoint inhibitor', 'anti-PD-1'),
    'chemotherapy': ('chemotherapy', 'unspecified'),
    'keytruda 25': ('checkpoint inhibitor', 'anti-PD-1'),
    'imfinzi 50': ('checkpoint inhibitor', 'anti-PD-L1'),
    'n-803': ('cytokine therapy', 'IL-15 superagonist'),
    'aldesleukin': ('cytokine therapy', 'IL-2'),
    'fluorouracil': ('chemotherapy', 'antimetabolite'),
    'methotrexate': ('chemotherapy', 'antifolate'),
    'yervoy 5': ('checkpoint inhibitor', 'anti-CTLA-4'),
    'prednisone': ('supportive care', 'steroid'),
    'dexamethasone': ('supportive care', 'steroid'),
    'tecentriq 1 200 concentrate for solution': ('checkpoint inhibitor', 'anti-PD-L1'),
    'cytarabine': ('chemotherapy', 'antimetabolite'),
    'pd-l1 t-hank': ('cell therapy', 'engineered T cell'),
    'fludarabine phosphate': ('chemotherapy', 'purine analog'),
    '5-fluorouracil': ('chemotherapy', 'antimetabolite'),
    'leucovorin': ('supportive care', 'folinic acid'),
    'interleukin-2': ('cytokine therapy', 'IL-2'),
    'mercaptopurine': ('chemotherapy', 'purine analog'),
    'sacituzumab govitecan': ('antibody-drug conjugate', 'anti-Trop-2'),
    'avastin 25': ('antibody', 'anti-VEGF'),
    'ceralasertib': ('targeted therapy', 'ATR inhibitor'),
    'filgrastim': ('supportive care', 'G-CSF'),
    'dacarbazine': ('chemotherapy', 'alkylating agent'),
    'tecentriq 1 200 concentrate for solution': ('checkpoint inhibitor', 'anti-PD-L1'),
    'tecentriq 840 concentrate for solution f': ('checkpoint inhibitor', 'anti-PD-L1'),
    'jemperli 500 concentrate for solution fo': ('checkpoint inhibitor', 'anti-PD-1'),
    'libtayo 350 concentrate for solution for': ('checkpoint inhibitor', 'anti-PD-1'),
    'anti-pd-1 monoclonal antibody': ('checkpoint inhibitor', 'anti-PD-1'),
    'pd-1 inhibitor': ('checkpoint inhibitor', 'anti-PD-1'),
    'pd-1': ('checkpoint inhibitor', 'anti-PD-1'),
    'palbociclib': ('targeted therapy', 'CDK4/6 inhibitor'),
    'ipatasertib': ('targeted therapy', 'AKT inhibitor'),
    'venetoclax': ('targeted therapy', 'BCL-2 inhibitor'),
    'rilvegostomig': ('checkpoint inhibitor', 'PD-1/CTLA-4 bispecific'),
    'bdb001': ('immunotherapy', 'STING agonist'),
    'prednisolone': ('supportive care', 'steroid'),
    'ifosfamide': ('chemotherapy', 'alkylating agent'),
    '5-fu': ('chemotherapy', 'antimetabolite'),
    'folinic acid': ('supportive care', 'rescue agent'),
    'abemaciclib': ('targeted therapy', 'CDK4/6 inhibitor'),
    'enfortumab vedotin': ('antibody-drug conjugate', 'anti-Nectin-4'),
    'imjudo 20': ('checkpoint inhibitor', 'anti-CTLA-4'),
    'prednisolone sodium succinate': ('supportive care', 'steroid'),
    'cytoxan': ('chemotherapy', 'alkylating agent'),
    'valacyclovir': ('antiviral', 'nucleoside analog'),
    'idarubicin': ('chemotherapy', 'anthracycline'),
    'il-2': ('cytokine therapy', 'IL-2'),
    'aglatimagene besadenovec': ('gene therapy', 'adenoviral vector'),
    'sx-682': ('targeted therapy', 'CXCR1/2 inhibitor'),
    'bendamustine': ('chemotherapy', 'alkylating agent'),
    'epirubicin': ('chemotherapy', 'anthracycline'),
    'tumor infiltrating lymphocytes': ('cell therapy', 'TILs'),
    'calcium folinate': ('supportive care', 'rescue agent'),
    'tecentriq 840 concentrate for solution f': ('checkpoint inhibitor', 'anti-PD-L1'),
    'jemperli 500 concentrate for solution fo': ('checkpoint inhibitor', 'anti-PD-1'),
    'libtayo 350 concentrate for solution for': ('checkpoint inhibitor', 'anti-PD-1'),
    'imjudo 20': ('checkpoint inhibitor', 'anti-CTLA-4'),
    'rt': ('radiotherapy', 'unspecified'),
    'radiation therapy': ('radiotherapy', 'unspecified'),
    'brachytherapy': ('radiotherapy', 'brachytherapy'),
    'androgen deprivation therapy': ('hormonal therapy', 'ADT'),
    'hormonal therapy': ('hormonal therapy', 'unspecified'),
    'anti-androgen therapy': ('hormonal therapy', 'anti-androgen'),
    'avastin 25': ('antibody', 'anti-VEGF'),
    'keytruda 25': ('checkpoint inhibitor', 'anti-PD-1'),
    'opdivo 10': ('checkpoint inhibitor', 'anti-PD-1'),
    'yervoy 5': ('checkpoint inhibitor', 'anti-CTLA-4'),
    'neoadjuvant chemotherapy': ('chemotherapy', 'neoadjuvant'),
    'concurrent chemotherapy': ('chemotherapy', 'concurrent'),
    'immunotherapy': ('immunotherapy', 'unspecified'),
    'chemotherapy': ('chemotherapy', 'unspecified'),
}

# FUNCTION TO CLASSIFY
def classify_drug(name):
    name = name.lower().strip()

    if any(chemo in name for chemo in chemo_agents):
        return 'chemotherapy', 'cytotoxic agent'
    if any(ci in name for ci in checkpoint_inhibitors):
        return 'checkpoint inhibitor', 'anti-PD-1/PD-L1'
    if any(ctla in name for ctla in ctla4_inhibitors):
        return 'checkpoint inhibitor', 'anti-CTLA-4'
    if any(parp in name for parp in parp_inhibitors):
        return 'targeted therapy', 'PARP inhibitor'
    if any(ot in name for ot in other_targeted):
        return 'targeted therapy', 'other'
    if name.endswith(mab_suffix):
        return 'antibody', 'mAb'
    if name.endswith(nib_suffix):
        return 'kinase inhibitor', 'TKI'
    if 'vaccine' in name:
        subtype = 'mRNA' if 'mrna' in name else 'vaccine'
        return 'vaccine', subtype
    if 'cart' in name or 'car-t' in name or 'tcr' in name:
        return 'cell therapy', 'engineered cell'
    if 'cell' in name:
        return 'cell therapy', 'other'
    if any(kw in name for kw in ['tang', 'herb', 'kampo', 'traditional']):
        return 'traditional medicine', None
    if name in ['standard of care', 'soc', 'best supportive care', 'supportive care']:
        return 'non-therapeutic', 'supportive'
    if any(term in name for term in surgery_terms):
        return 'surgery', 'procedure'
    return 'other', None


# MAIN PARSING FUNCTION & CLASSIFICATION
def clean_intervention_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\b(ml|mg|mcg|g|%)\b', '', text)
    text = re.sub(r'\b(intratumoral|iv|subcutaneous|oral|infusion|administration|placebo)\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def parse_intervention(text):
    if pd.isna(text):
        return [], []
    
    text = text.lower()
    text = clean_intervention_text(text)
    interventions = []

    # Handle CTG/CTIS tagged format
    if any(tag in text for tag in ['drug:', 'biological:', 'combination_product:', '|']):
        blocks = re.split(r'\s*\|\s*', text)
        for block in blocks:
            match = re.match(r'(drug|biological|combination_product):\s*(.+)', block.strip())
            if match:
                _, content = match.groups()
                content_cleaned = clean_intervention_text(content)
                candidates = re.split(r'[,+/]', content_cleaned)
                interventions.extend([c.strip() for c in candidates if c.strip()])
    else:
        interventions = [x.strip() for x in re.split(r'[,+/]', text) if x.strip()]

    # Classification step
    types = set()
    details = []

    for raw in interventions:
        main_type, subtype = classify_drug(raw)
        types.add(main_type)
        details.append({'type': main_type, 'subtype': subtype, 'raw': raw})

    return list(types), details

def apply_manual_classification(details_list):
    corrected = []
    for d in details_list:
        raw = d['raw'].lower().strip()
        if d['type'] == 'other' and raw in manual_map:
            new_type, new_subtype = manual_map[raw]
            corrected.append({'type': new_type, 'subtype': new_subtype, 'raw': raw})
        else:
            corrected.append(d)
    return corrected

# APPLYING TO DATAFRAME
df[['intervention_group', 'intervention_details']] = df['intervention'].apply(parse_intervention).apply(pd.Series)
df['intervention_details'] = df['intervention_details'].apply(apply_manual_classification)
df['intervention_group'] = df['intervention_details'].apply(lambda lst: list({d['type'] for d in lst}))

# CLASSIFIED AS OTHERS...

generic_terms = [
    'concentrate for solution for', 'concentrate for solution for .',
    'konzentrat zur herstellung einer infusion',
    'solution à diluer pour perfusion',
    'concentraat voor oplossing voor infusie',
    'immunotherapy', 'chemotherapy', 'treatment',
    'anti-pd-1 monoclonal antibody', 'pd-1 antibody',
    'neoadjuvant chemotherapy'
]

def get_title_temp(trial_id):
    if trial_id in set(df1['id']):
        return df1.loc[df1['id'] == trial_id, 'Study Title'].values[0]
    else:
        return df2.loc[df2['id'] == trial_id, 'Title of the trial'].values[0]

def update_intervention_details(row):
    trial_id = row['id']
    title = get_title_temp(trial_id).lower()
    new_details = []
    modified = False

    for d in row['intervention_details']:
        raw_clean = clean_intervention_text(d['raw'].lower())

        if d['type'] == 'other' and raw_clean in generic_terms:
            # Checkpoint inhibitors
            if any(drug in title for drug in checkpoint_inhibitors):
                match = next(drug for drug in checkpoint_inhibitors if drug in title)
                new_details.append({'type': 'checkpoint inhibitor', 'subtype': 'anti-PD-1/PD-L1 (title)', 'raw': match})
                modified = True
            # CTLA-4
            elif any(ctla in title for ctla in ctla4_inhibitors):
                match = next(drug for drug in ctla4_inhibitors if drug in title)
                new_details.append({'type': 'checkpoint inhibitor', 'subtype': 'anti-CTLA-4 (title)', 'raw': match})
                modified = True
            # Radiation
            elif 'radiotherapy' in title or 'radiation' in title or 'brachytherapy' in title:
                new_details.append({'type': 'radiotherapy', 'subtype': 'from title', 'raw': 'radiotherapy'})
                modified = True
            # Hormonal
            elif 'androgen deprivation' in title or 'hormonal therapy' in title or 'anti-androgen' in title:
                new_details.append({'type': 'hormonal therapy', 'subtype': 'from title', 'raw': 'hormonal therapy'})
                modified = True
            else:
                new_details.append(d)
        else:
            new_details.append(d)

    if modified:
        row['intervention_group'] = list({d['type'] for d in new_details})
    return new_details

df['intervention_details'] = df.apply(update_intervention_details, axis=1)
df['intervention_group'] = df['intervention_details'].apply(lambda lst: list({d['type'] for d in lst}))


# TABLES AND GRAPHS 
df_exploded = df.explode('intervention_group')

interv_counts = df_exploded['intervention_group'].value_counts()
total = interv_counts.sum()
total_unique = interv_counts.shape[0]

df['is_combination'] = df['intervention_group'].apply(lambda x: len(x) > 1)
n_combination = df['is_combination'].sum()
pct_combination = n_combination / len(df) * 100

table_str = (
    f"{'Intervention Group':30} | {'Count':>5} | {'%':>5}\n"
    + "-"*50 + "\n"
)
for name, count in interv_counts.items():
    pct = (count / total * 100)
    table_str += f"{name[:30]:30} | {count:5d} | {pct:5.1f}\n"

table_str += "-"*50 + "\n"
table_str += f"{'TOTAL UNIQUE':30} | {total_unique:5d} |\n"
table_str += f"{'Combination therapy':30} | {n_combination:5d} | {pct_combination:5.1f}\n"

print(table_str)



interv_summary = interv_counts.reset_index()
interv_summary.columns = ['Intervention Group', 'Count']


interv_summary = pd.concat([
    interv_summary,
    pd.DataFrame([{'Intervention Group': 'Combination therapy', 'Count': n_combination}])
], ignore_index=True)

plt.figure(figsize=(10,6))
sns.barplot(
    data=interv_summary,
    x="Intervention Group",
    y="Count",
    palette="Blues_d"
)
plt.xticks(rotation=30, ha="right")
plt.title("Distribution of Clinical Trials by Intervention Group", fontsize=14, weight="bold")
plt.ylabel("Number of Trials")
plt.xlabel("")
plt.tight_layout()
plt.show()


In [ ]:
# SUMMARY OF CONDITIONS

# ClEANING THE RAW TEXT
import re

def clean_condition_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)         # remove content in parentheses
    text = re.sub(r'[^a-z0-9\s]', ' ', text)      # remove special characters
    text = re.sub(r'\s+', ' ', text).strip()      # normalize whitespace
    return text

df['condition_clean_text'] = df['condition'].apply(clean_condition_text)

# KEYWORD MAP
condition_map = {
    "gynecologic": [
        "cervical", "endometrial", "ovarian", "uterine", "vaginal", "vulvar",
        "gynaecological", "gynecologic"
    ],
    "genitourinary": [
        "prostate", "prostatic", "bladder", "urothelial", "renal", "kidney", 
        "testicular", "penile", "urinary"
    ],
    "digestive": [
        "colorectal", "colon", "rectal", "gastric", "stomach", "pancreatic",
        "esophageal", "liver", "hepatocellular", "biliary", "anal", "gastrointestinal"
    ],
    "lung/thoracic": [
        "lung", "nsclc", "small cell", "mesothelioma", "pleural", "thoracic",
        "pulmonary", "bronchial"
    ],
    "breast": ["breast"],
    "hematologic": [
        "leukemia", "lymphoma", "myeloma", "hematologic", "aml", "cll", "mcl", 
        "mm", "b-cell", "t-cell", "myelodysplastic", "mpn"
    ],
    "skin": ["melanoma", "skin", "cutaneous", "merkel"],
    "brain/CNS": [
        "glioblastoma", "glioma", "astrocytoma", "cns", "brain", "medulloblastoma",
        "ependymoma", "neurological"
    ],
    "head/neck": [
        "head", "neck", "oropharyngeal", "nasopharyngeal", "oral", "laryngeal",
        "pharyngeal", "tongue"
    ],
    "sarcoma": [
        "sarcoma", "osteosarcoma", "rhabdomyosarcoma", "chondrosarcoma", "angiosarcoma"
    ],
    "multiple": [
        "advanced cancer", "advanced malignancy", "solid tumor", "metastatic",
        "locally advanced", "multiple tumor", "refractory", "relapsed"
    ],
    "other": []
}


# FUNCTION TO CLASSIFY
def classify_condition(text):
    if pd.isna(text):
        return "unknown"
    text_lower = text.lower()
    for group, keywords in condition_map.items():
        if any(keyword in text_lower for keyword in keywords):
            return group
    return "other"

# APPLYING TO DATAFRAME
df['condition_group'] = df['condition'].apply(classify_condition)

# TABLE & GRAPHS
summary_table = df['condition_group'].value_counts(normalize=False).reset_index()
summary_table.columns = ["Condition Group", "Count"]
summary_table["Percentage"] = (summary_table["Count"] / summary_table["Count"].sum() * 100).round(1)
print(summary_table)

plt.figure(figsize=(10,6))
sns.barplot(
    data=summary_table,
    x="Condition Group",
    y="Count",
    palette="Blues_d"
)
plt.xticks(rotation=30, ha="right")
plt.title("Distribution of Clinical Trials by Condition Group", fontsize=14, weight="bold")
plt.ylabel("Number of Trials")
plt.xlabel("")
plt.tight_layout()
plt.show()


In [ ]:
# SUMMARY OF SPONSOR TYPE DISTRIBUTION

sponsors = df['sponsor_type'].value_counts().reset_index()
sponsors.columns = ['sponsor_type', 'count']

def clean_sponsor_type(s):
    if pd.isna(s):
        return "Other"
    parts = [part.strip() for part in s.split(",")]
    unique_parts = list(dict.fromkeys(parts))
    first = unique_parts[0]
    return first

df['sponsor_type_clean'] = df['sponsor_type'].apply(clean_sponsor_type)

def regroup_category(s):
    if s in ["NIH", "FED", "OTHER_GOV"]:
        return "Government"
    elif s in ["INDUSTRY", "Pharmaceutical company"]:
        return "Industry"
    elif "Hospital/Clinic" in s:
        return "Hospital/Clinic"
    elif s == "Patient organisation/association":
        return "Patient Organisation"
    elif s == "NETWORK":
        return "Network"
    elif s in ["Educational Institution", "Laboratory/Research/Testing facility"]:
        return "Academic"
    elif s == "OTHER":
        return "Other"
    else:
        return "Other"

df['sponsor_type_grouped'] = df['sponsor_type_clean'].apply(regroup_category)


sponsor_group_counts = df['sponsor_type_grouped'].value_counts()
total_trials = sponsor_group_counts.sum()
table_str = (
    f"{'Sponsor Type':25} | {'Count':5} | {'% of Total'}\n"
    + "-"*55 + "\n"
)
for name, count in sponsor_group_counts.items():
    pct = round(count / total_trials * 100, 1)
    table_str += f"{name:25} | {count:<5} | {pct:.1f}%\n"
table_str += "-"*55 + "\n"
table_str += f"{'TOTAL':25} | {total_trials:<5} | 100%\n"
print(table_str)



plt.figure(figsize=(10,6))
sns.barplot(
    y=sponsor_group_counts.index,
    x=sponsor_group_counts.values,
    palette="Blues_d"
)
plt.title("Grouped Sponsor Types", fontsize=14, weight="bold")
plt.xlabel("Number of Trials")
plt.ylabel("")
plt.tight_layout()
plt.show()


In [ ]:

# SUMMARY OF SPONSORS 

sponsor_counts = df['sponsor'].value_counts()


total_unique = sponsor_counts.shape[0]
total_counts = sponsor_counts.sum()

table_str = (
    f"{'Sponsor':50} | {'Count':5} | {'% of Total'}\n"
    + "-"*80 + "\n"
)

for name, count in sponsor_counts.items():
    pct = round(count / total_counts * 100, 1)
    table_str += f"{name[:50]:50} | {count:<5} | {pct:.1f}%\n"

table_str += "-"*80 + "\n"
table_str += f"{'TOTAL UNIQUE':50} | {total_counts:<5} | 100%\n"

print(table_str)


In [ ]:
# SUMMARY OF COUNTRY DISTRIBUTION

country_list = []

for loc in df['location']:
    if pd.notna(loc):
        countries = [c.strip() for c in loc.split(",")]
        country_list.extend(countries)

countries_series = pd.Series(country_list)
countries_counts = countries_series.value_counts()

print("== Countries hosting trials ==")
print(countries_counts)

# PIE CHART (3% threshold)
countries_pct = countries_series.value_counts(normalize=True) * 100
threshold = 3
large_countries = countries_pct[countries_pct >= threshold]
small_countries = countries_pct[countries_pct < threshold]
others = small_countries.sum()
final_countries = large_countries.copy()
if others > 0:
    final_countries["Others"] = others

import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
plt.pie(
    final_countries,
    labels=final_countries.index,
    autopct='%1.1f%%',
    startangle=140,
    wedgeprops={"edgecolor":"white"}
)
plt.title("Clinical Trials by Country", fontsize=14, weight="bold")
plt.tight_layout()
plt.show()


In [ ]:
# SUMMARY OF PHASES DISTRIBIUTION 

phase_labels = {
    1: "Phase I",
    2: "Phase II",
    3: "Phase III",
    12: "Phase I & II",
    23: "Phase II & III"
}

df['phase_named'] = df['phase'].map(phase_labels)
phase_counts = df['phase_named'].value_counts().reindex(
    ["Phase I", "Phase II", "Phase III", "Phase I & II", "Phase II & III"]
).fillna(0)

plt.figure(figsize=(8,5))
sns.barplot(
    x=phase_counts.index,
    y=phase_counts.values,
    palette="Purples_d"
)

plt.title("Clinical Trial Phase Distribution", fontsize=14, weight="bold")
plt.ylabel("Number of Trials")
plt.xlabel("")
plt.tight_layout()
plt.show()


In [ ]:
# FINAL DATA SET TO CSV
df.to_csv("final_dataset_studies.csv", index=False)